In [ ]:
import string

def caesar_cipher_encrypt(text, shift):
    result = ""
    for char in text:
        if char.isalpha():
            if char.isupper():
                result += chr((ord(char) - 65 + shift) % 26 + 65)
            else:
                result += chr((ord(char) - 97 + shift) % 26 + 97)
        else:
            result += char
    return result

def caesar_cipher_decrypt(ciphertext, shift):
    return caesar_cipher_encrypt(ciphertext, -shift)

def monoalphabetic_cipher_encrypt(text, key):
    result = ""
    for c in text:
        if c.isalpha():
            if c.isupper():
                result += key[ord(c) - ord('A')].upper()
            else:
                result += key[ord(c) - ord('a')].lower()
        else:
            result += c
    return result

def monoalphabetic_cipher_decrypt(ciphertext, key):
    inverse_key = ''.join(sorted(string.ascii_lowercase, key=lambda x: key.index(x)))
    return monoalphabetic_cipher_encrypt(ciphertext, inverse_key)

def generate_key_square(keyword):
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
    key_square = ""

    # Append the keyword to the key square
    for char in keyword.upper():
        if char not in key_square and char in alphabet:
            key_square += char

    # Append the remaining alphabet characters to the key square
    for char in alphabet:
        if char not in key_square:
            key_square += char

    # Convert the key square string to a 5x5 matrix
    key_square_matrix = [list(key_square[i:i+5]) for i in range(0, 25, 5)]

    return key_square_matrix

def encrypt_playfair(plaintext, key_square):
    plaintext = plaintext.upper().replace('J', 'I').replace(' ', '')
    plaintext_pairs = [plaintext[i:i + 2] for i in range(0, len(plaintext), 2)]

    encrypted_text = ""
    for pair in plaintext_pairs:
        if len(pair) == 1:
            pair += 'X'

        char1, char2 = pair[0], pair[1]
        row1, col1 = get_coordinates(key_square, char1)
        row2, col2 = get_coordinates(key_square, char2)

        if row1 == row2:
            encrypted_text += key_square[row1][(col1 + 1) % 5] + key_square[row2][(col2 + 1) % 5]
        elif col1 == col2:
            encrypted_text += key_square[(row1 + 1) % 5][col1] + key_square[(row2 + 1) % 5][col2]
        else:
            encrypted_text += key_square[row1][col2] + key_square[row2][col1]

    return encrypted_text

def decrypt_playfair(ciphertext, key_square):
    decrypted_text = ""
    for i in range(0, len(ciphertext), 2):
        char1, char2 = ciphertext[i], ciphertext[i + 1]
        row1, col1 = get_coordinates(key_square, char1)
        row2, col2 = get_coordinates(key_square, char2)

        if row1 == row2:
            decrypted_text += key_square[row1][(col1 - 1) % 5] + key_square[row2][(col2 - 1) % 5]
        elif col1 == col2:
            decrypted_text += key_square[(row1 - 1) % 5][col1] + key_square[(row2 - 1) % 5][col2]
        else:
            decrypted_text += key_square[row1][col2] + key_square[row2][col1]

    return decrypted_text.lower()

def rail_fence_cipher_encrypt(plain_text, key):
    # Strip blank spaces from the input
    plain_text = plain_text.replace(' ', '')

    # Create the matrix to cipher plain text
    rail = [[' ' for _ in range(len(plain_text))] for _ in range(key)]

    # To find the direction
    dir_down = False
    row = 0
    col = 0

    for char in plain_text:
        # Check if character is not a space
        if char != ' ':
            # Check the direction of flow
            # Reverse the direction if we've just
            # filled the top or bottom rail
            if row == 0 or row == key - 1:
                dir_down = not dir_down

            # Fill the corresponding alphabet
            rail[row][col] = char
            col += 1

            # Find the next row using direction flag
            if dir_down:
                row += 1
            else:
                row -= 1

    # Construct the cipher using the rail matrix
    result = ''.join(''.join(row) for row in rail)
    return result

def rail_fence_cipher_decrypt(ciphertext, key):
    # Strip blank spaces from the input
    ciphertext = ciphertext.replace(' ', '')

    # Create the matrix to cipher plain text
    rail = [[' ' for _ in range(len(ciphertext))] for _ in range(key)]

    # To find the direction
    dir_down = None
    row = 0
    col = 0

    # Mark the places with '*'
    for _ in range(len(ciphertext)):
        # Check if character is not a space
        if ciphertext[_] != ' ':
            # Check the direction of flow
            if row == 0:
                dir_down = True
            if row == key - 1:
                dir_down = False

            # Place the marker
            rail[row][col] = '*'

            # Find the next row using direction flag
            if dir_down:
                row += 1
            else:
                row -= 1
            col += 1

    # Now we can construct and fill the rail matrix
    index = 0
    for i in range(key):
        for j in range(len(ciphertext)):
            if rail[i][j] == '*' and index < len(ciphertext):
                rail[i][j] = ciphertext[index]
                index += 1

    # Read the matrix in zig-zag manner to construct the resultant text
    result = ''
    row = 0
    col = 0
    for _ in range(len(ciphertext)):
        # Check if character is not a space
        if ciphertext[_] != ' ':
            # Check the direction of flow
            if row == 0:
                dir_down = True
            if row == key - 1:
                dir_down = False

            # Append the character if it's not '*'
            if rail[row][col] != '*':
                result += rail[row][col]
                col += 1

            # Find the next row using direction flag
            if dir_down:
                row += 1
            else:
                row -= 1

    return result

def get_coordinates(key_square, char):
    for row in range(5):
        for col in range(5):
            if key_square[row][col] == char:
                return row, col
    raise ValueError("Character not found in the key square")

def main():
    while True:
        plain_text = input("Enter the plaintext: ").strip()  # Strip blank spaces from the input

        print("Choose a cipher:")
        print("1. Caesar Cipher")
        print("2. Monoalphabetic Cipher")
        print("3. Playfair Cipher")
        print("4. Rail Fence Cipher")
        print("5. Exit")

        choice = input("Enter your choice (1/2/3/4/5): ")

        if choice == '5':
            break

        if choice == '1':
            shift_caesar = int(input("Enter the shift value for Caesar Cipher: "))
            cipher_text = caesar_cipher_encrypt(plain_text, shift_caesar)
            print("Caesar Cipher (Encrypted Text):", cipher_text)
            decrypt_choice = input("Do you want to decrypt the ciphertext? (yes/no): ").lower()
            if decrypt_choice == 'yes':
                decrypted_text = caesar_cipher_decrypt(cipher_text, shift_caesar)
                print("Caesar Cipher (Decrypted Text):", decrypted_text)
        elif choice == '2':
            key_monoalphabetic = input("Enter the key for Monoalphabetic Cipher (a permutation of the alphabet): ")
            cipher_text = monoalphabetic_cipher_encrypt(plain_text, key_monoalphabetic)
            print("Monoalphabetic Cipher (Encrypted Text):", cipher_text)
            decrypt_choice = input("Do you want to decrypt the ciphertext? (yes/no): ").lower()
            if decrypt_choice == 'yes':
                decrypted_text = monoalphabetic_cipher_decrypt(cipher_text, key_monoalphabetic)
                print("Monoalphabetic Cipher (Decrypted Text):", decrypted_text)
        elif choice == '3':
            key_playfair = input("Enter the key for Playfair Cipher (a keyword): ")
            key_square = generate_key_square(key_playfair)
            cipher_text = encrypt_playfair(plain_text, key_square)
            print("Playfair Cipher (Encrypted Text):", cipher_text)
            decrypt_choice = input("Do you want to decrypt the ciphertext? (yes/no): ").lower()
            if decrypt_choice == 'yes':
                decrypted_text = decrypt_playfair(cipher_text, key_square)
                print("Playfair Cipher (Decrypted Text):", decrypted_text)
        elif choice == '4':
            key_rail_fence = int(input("Enter the key for Rail Fence Cipher (number of rails): "))
            cipher_text = rail_fence_cipher_encrypt(plain_text, key_rail_fence)
            print("Rail Fence Cipher (Encrypted Text):", cipher_text)
            decrypt_choice = input("Do you want to decrypt the ciphertext? (yes/no): ").lower()
            if decrypt_choice == 'yes':
                decrypted_text = rail_fence_cipher_decrypt(cipher_text, key_rail_fence)
                print("Rail Fence Cipher (Decrypted Text):", decrypted_text)
        else:
            print("Invalid choice. Please enter 1, 2, 3, 4, or 5.")

        run_again = input("Do you want to continue? (yes/no): ").lower()
        if run_again != 'yes':
            break

if __name__ == "__main__":
    main()


Enter the plaintext: rainbow
Choose a cipher:
1. Caesar Cipher
2. Monoalphabetic Cipher
3. Playfair Cipher
4. Rail Fence Cipher
5. Exit
Enter your choice (1/2/3/4/5): 5
